In [2]:
import json
import requests

In [21]:
#Formatted SoQL url to get all data from API by Surgical HCPCS (10004 - 69990)
url = "https://data.cms.gov/resource/2zuc-y5mm.json?$where=hcpcs_code between '10004' and '69990'&$limit=10000 "
request = requests.get(url)
data = request.json()



# Medicare Data Set of Surgical HCPCS Codes Reported on Claims - 2017

* CMS API Endpoint: https://data.cms.gov/resource/2zuc-y5mm.json
* API Documentation: https://dev.socrata.com/foundry/data.cms.gov/2zuc-y5mm


## List of fields in Data Set:

* HCPCS Code	
* HCPCS Description	
* HCPCS Drug Indicator	
* Place of Service	
* Number of Providers	
* Number of Services	
* Number of Unique 
* Beneficiary/Provider Interactions	
* Number of Distinct Medicare Beneficiary/Per Day Services	
* Average Submitted Charge Amount	
* Average Medicare Allowed Amount	
* Average Medicare Payment Amount	
* Average Medicare Standardized Payment Amount


## What data do I need for HCPCS code Class? 

* Code ('hcpcs_code')
* Description ('hcpcs_description')
* Number of Services
* Average Charged Amount ('average_submitted_charge_amount'
* Average Allowed Amount ('average_medicare_allowed_amount': '120.05')
* Average Paid Amount ('average_medicare_standardized_payment_amount)
* Drug ('hcpcs_drug_indicator')


In [23]:
class Procedure:
    def __init__(self, code, description, services, avg_charge, avg_allowed, avg_paid, drug):
        self.code = code
        self.description=description
        self.services=services
        self.avg_charge=avg_charge
        self.avg_allowed=avg_allowed
        self.drug=False
    

## Goals From Here:

* Formulate all data into HCPCS Code classes that can be used in lists (one for Office, one for Facility) that can then be analyzed.
* Analyze and visualize data using something like matplotlib to identify key data stories.


In [17]:
def get_totals (list, text, result):
    for i in list:
        amount = int(i["number_of_services"]) * float(i[text])
        result.append((i["hcpcs_code"], amount))
    

In [4]:
facility_data = []
office_data = []

for i in data:
    if i["place_of_service"] == "F":
        facility_data.append(i)
    elif i["place_of_service"] == "O":
        office_data.append(i)

In [20]:
facility_total_paid = []
get_totals(facility_data, "average_medicare_standardized_payment_amount", facility_total_paid)

facility_total_allowed = []
get_totals(facility_data, 'average_medicare_allowed_amount', facility_total_allowed)

office_total_paid = []
get_totals(office_data, "average_medicare_standardized_payment_amount", facility_total_paid)

office_total_allowed = []
get_totals(office_data, 'average_medicare_allowed_amount', office_total_allowed)
